<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/venv/notebooks/shakespeare_trainingV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from datetime import datetime
print("Current Date and Time:", datetime.now())

Current Date and Time: 2024-11-09 09:36:27.058039


In [1]:
%%time
%cd /content
# Don't use --depth 1 because that does not play nice with git-annex
!git clone -b venv https://github.com/markNZed/GPT-NeoX-Colab.git
%cd /content/GPT-NeoX-Colab
!pip install . > /dev/null 2>&1
from dotenv import load_dotenv
import os
load_dotenv('/content/GPT-NeoX-Colab/.env')
import GPTNeoXColab
GPTNeoXColab.utils.colab.install_git_annex()
GPTNeoXColab.utils.colab.enable_remote()
GPTNeoXColab.utils.colab.sync_annex()
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare.txt")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare.jsonl")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare_text_document.bin")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare_text_document.idx")

/content
Cloning into 'GPT-NeoX-Colab'...
remote: Enumerating objects: 839, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 839 (delta 85), reused 157 (delta 61), pack-reused 640 (from 1)
Receiving objects: 100% (839/839), 15.56 MiB | 1.69 MiB/s, done.
Resolving deltas: 100% (423/423), done.
/content/GPT-NeoX-Colab
git-annex is already installed.
Starting git annex sync...
Sync successful. Fetching data from backblaze...
Data retrieval successful.
Data retrieval successful.
Data retrieval successful.
Data retrieval successful.
CPU times: user 10.4 s, sys: 1.26 s, total: 11.6 s
Wall time: 41 s


In [21]:
#@title Mount Google Drive
GPTNeoXColab.utils.colab.mount_google_drive(False)

In [22]:
#@title SSH Connection for VS Code
GPTNeoXColab.utils.colab.setup_ssh_connection(False)

This function requires Google Colab and won't work in other environments.


# Cloning GPT-NeoX Repo

In [23]:
%%time
%cd /content
!git clone --depth 1 https://github.com/EleutherAI/gpt-neox

/content
Cloning into 'gpt-neox'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 296 (delta 74), reused 136 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (296/296), 2.50 MiB | 2.44 MiB/s, done.
Resolving deltas: 100% (74/74), done.
CPU times: user 60.8 ms, sys: 21.8 ms, total: 82.5 ms
Wall time: 2 s


In [24]:
%%time
%cd /content
BACKBLAZE_SAVE = False
GPTNeoXColab.utils.colab.download_my_env(BACKBLAZE_SAVE)

/content
Unzipping my_env.tar.gz


Exception: Command failed with return code 127 CompletedProcess(args='pigz -d -p 4 /content/my_env.tar.gz', returncode=127, stdout=b'', stderr=b'/bin/sh: 1: pigz: not found\n')

# Installing Dependencies

In [25]:
%%time
%cd /content
import os
# Check if the directory does not exist
if not os.path.isdir("/content/my_env"):
    # Install venv package for Python 3.10
    !apt-get update && apt-get install -y python3.10-venv
    !pip install virtualenv
    # Create the virtual environment
    !python3 -m venv /content/my_env
    %cd /content/gpt-neox
    # Install specific versions of torch and other packages to avoid compatibility issues
    !source /content/my_env/bin/activate && pip install torch==2.3.0 torchaudio==2.3.0 torchvision==0.18.0 transformers==4.41.0 sentence-transformers==2.2.2
    # Install dependencies
    !source /content/my_env/bin/activate && pip install -r ./requirements/requirements.txt
    !source /content/my_env/bin/activate && pip install -r ./requirements/requirements-tensorboard.txt

/content


Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
Defaulting to user installation because normal site-packages is not writeable
/content/gpt-neox
/bin/sh: 1: source: not found
/bin/sh: 1: source: not found
/bin/sh: 1: source: not found
CPU times: user 349 ms, sys: 85.5 ms, total: 434 ms
Wall time: 15.3 s


In [26]:
GPTNeoXColab.utils.colab.upload_my_env(BACKBLAZE_SAVE)

# Preparing Custom Dataset

In [27]:
import os

%cd /content/gpt-neox
!mkdir -p data

# Check if the file exists
if not os.path.isfile("/content/GPT-NeoX-Colab/data/shakespeare.jsonl"):
    GPTNeoXColab.utils.ml.text2jsonl("/content/GPT-NeoX-Colab/data/shakespeare.jsonl", '/content/GPT-NeoX-Colab/data/shakespeare.jsonl')


!cp /content/GPT-NeoX-Colab/data/shakespeare.jsonl /content/GPT-NeoX-Colab/data/shakespeare.jsonl

/content/gpt-neox


AttributeError: module 'GPTNeoXColab.utils' has no attribute 'ml'

# TinyShakespeare Dataset

# Tokenizing Dataset

In [16]:
%%time
%%bash
source /content/my_env/bin/activate
cd /content/gpt-neox
mkdir -p processed_data
python tools/datasets/preprocess_data.py \
    --input ./data/shakespeare.jsonl \
    --output-prefix ./processed_data/shakespeare \
    --tokenizer-type CharLevelTokenizer \
    --dataset-impl mmap \
    --append-eod

Traceback (most recent call last):
  File "/content/gpt-neox/tools/datasets/preprocess_data.py", line 25, in <module>
    import lm_dataformat as lmd
ModuleNotFoundError: No module named 'lm_dataformat'


CalledProcessError: Command 'b'source /content/my_env/bin/activate\ncd /content/gpt-neox\nmkdir -p processed_data\npython tools/datasets/preprocess_data.py \\\n    --input ./data/shakespeare.jsonl \\\n    --output-prefix ./processed_data/shakespeare \\\n    --tokenizer-type CharLevelTokenizer \\\n    --dataset-impl mmap \\\n    --append-eod\n'' returned non-zero exit status 1.

# Training

In [ ]:
%load_ext tensorboard

In [ ]:
import subprocess

# Start a detached background process
process = subprocess.Popen(
    "nohup bash -c \"source /content/my_env/bin/activate && python ./deepy.py train.py --conf_dir /content/GPT-NeoX-Colab/configs shakespeare.yml shakespeare_deepy.yml\" & echo $! > train_process.pid",
    shell=True,
    executable='/bin/bash',
    preexec_fn=subprocess.os.setsid  # Starts the process in a new session so interrupting Notebook does not kill the training
)


In [ ]:
# Wait until a tensorboard is created
import time
import os

# Path to the TensorBoard log directory
tensorboard_log_dir = '/content/gpt-neox/tensorboard'

# Wait for the directory to be created
while not os.path.exists(tensorboard_log_dir):
    print("Waiting for TensorBoard log directory to be created...")
    time.sleep(10)  # Check every X seconds

print("TensorBoard log directory found. You can now launch TensorBoard.")


In [ ]:
# Need to remove everything in checkpoints and tensorboard dir for a fresh run
%tensorboard --logdir tensorboard

In [ ]:
import glob
import os

# Define the log directory and pattern for log files
log_dir = "/content/gpt-neox/logs"
log_pattern = os.path.join(log_dir, "*_stdout.txt")

# Get the list of log files that match the pattern
log_files = glob.glob(log_pattern)

# Ensure there are log files in the directory
if log_files:
    # Find the latest log file based on modification time
    latest_log = max(log_files, key=os.path.getmtime)
    print("Latest log file:", latest_log)
else:
    latest_log = None
    print("No log files found.")

# Now `latest_log` holds the path to the latest log file or None if not found


In [ ]:
import time
import os
import re

# File to store the last read position (persistence between script runs)
file_position = 0
# Regular expression to match "iteration <number> / <total>"
iteration_pattern = re.compile(r"iteration\s+(\d+)\s*/\s*\d+")

def read_new_iterations():
    global file_position
    # Open the log file and seek to the last position
    with open(latest_log, "r") as file:
        file.seek(file_position)
        # Read new lines
        new_lines = file.readlines()
        file_position = file.tell()
        # Process lines containing "iteration"
        last_match = None
        for line in new_lines:
            match = iteration_pattern.search(line)
            if match:
                last_match = match
        if last_match:
            # Extract the iteration count from the regex match
            iteration_count = int(last_match.group(1))
            print(f"{iteration_count} iterations")

# Read the PID from the file
with open("train_process.pid", "r") as f:
    pid = int(f.read().strip())
    print("PID:", pid)

# Function to check if the process is running
def is_process_running(pid):
    try:
        os.kill(pid, 0)  # Sending signal 0 to check if the process exists
        return True
    except OSError:
        return False

# Monitor the process
while is_process_running(pid):
    read_new_iterations()
    print("Training is still running...")
    time.sleep(30)  # Check every X seconds

print("Training has finished.")


In [ ]:
# Wait until a checkpoint is created
import time
import os

# Path to the checkpoints directory
checkpoints_dir = '/content/gpt-neox/checkpoints'

# Wait for the directory to be created
while not os.path.exists(checkpoints_dir):
    print("Waiting for checkpoints directory to be created...")
    time.sleep(10)  # Check every X seconds

print("Checkpoints directory found.")


In [ ]:
#@title Training and Validation Loss
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
# Path to the latest log file
log_dir = "tensorboard"
log_files = [os.path.join(log_dir, d) for d in os.listdir(log_dir)]
latest_log_dir = max(log_files, key=os.path.getmtime)

# Initialize EventAccumulator to load scalar data
ea = event_accumulator.EventAccumulator(latest_log_dir)
ea.Reload()  # Load all logs

# List all scalar keys available in the logs
scalar_keys = ea.Tags()['scalars']
print("Available scalar keys:", scalar_keys)

# Extract training and validation losses
train_loss = ea.Scalars('train/lm_loss')  # Adjust for actual name if necessary
val_loss = ea.Scalars('validation/lm_loss')  # Adjust for actual name if necessary

# Convert to lists for plotting
train_loss_values = [x.value for x in train_loss]
val_loss_values = [x.value for x in val_loss]

# Find the lengths of both arrays
len_train = len(train_loss_values)
len_val = len(val_loss_values)

iterations = None
# Interpolate the shorter array
if len_train != len_val:
    if len_train > len_val:
        # Interpolate validation loss to match the training loss length
        iterations = np.linspace(1, len_train, len_train)
        val_iterations = np.linspace(1, len_train, len_val)
        val_loss_values = np.interp(iterations, val_iterations, val_loss_values)
    else:
        # Interpolate training loss to match the validation loss length
        iterations = np.linspace(1, len_val, len_val)
        train_iterations = np.linspace(1, len_val, len_train)
        train_loss_values = np.interp(iterations, train_iterations, train_loss_values)
else:
    iterations = range(1, len_train + 1)

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(iterations, train_loss_values, label='Training Loss')
plt.plot(iterations, val_loss_values, label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# If we want to monitor the logs
#!tail -f /content/gpt-neox/logs/*stdout.txt

# Inference with GPT-NeoX

In [ ]:
%%time
# This has issues if used during training -  The server socket has failed to bind to [::]:29500 (errno: 98 - Address already
# This will write over the logs
!source /content/my_env/bin/activate && python ./deepy.py generate.py -d configs /content/GPT-NeoX-Colab/configs/shakespeare.yml /content/GPT-NeoX-Colab/configs/shakespeare_gen.yml > /dev/null
!cat sample_output.txt

# Inference with Hugging Face

## Convert Our Model to HuggingFace Format
Here we are converting our model to `HuggingFace Format`.

In [ ]:
import os

# Define the path to the checkpoints directory
checkpoints_dir = "/content/gpt-neox/checkpoints"

# Read the 'latest' file to get the latest checkpoint name
with open(os.path.join(checkpoints_dir, "latest"), "r") as f:
    latest_checkpoint_name = f.read().strip()

# Construct the full path to the latest checkpoint directory
latest_checkpoint_path = os.path.join(checkpoints_dir, latest_checkpoint_name)
print("Path to the latest checkpoint:", latest_checkpoint_path)


In [ ]:
!source /content/my_env/bin/activate && python ./tools/ckpts/convert_neox_to_hf.py --input_dir {latest_checkpoint_path} --config_file /content/GPT-NeoX-Colab/configs/shakespeare.yml --output_dir hf_model/save/location --precision auto --architecture neox

## Generating Text

In [ ]:
from transformers import GPTNeoXForCausalLM
import torch

# Move to model directory
%cd /content/gpt-neox

# Assuming CharLevelTokenizer is properly imported and instantiated
from GPTNeoXColab import CharLevelTokenizer
tokenizer = CharLevelTokenizer.CharLevelTokenizer(vocab_size=512)

# Load your model
model_path = "/content/gpt-neox/hf_model/save/location"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Define a simple char-level tokenizer if not provided
def char_level_tokenize(text):
    return tokenizer.tokenize(text)

def char_level_detokenize(tokens):
    return tokenizer.detokenize(tokens)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
#input_text = input("Enter your prompt: ")
input_text = "Thou art"

# Tokenize and prepare input
input_ids = torch.tensor([char_level_tokenize(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode and print the generated text
generated_text = char_level_detokenize(output[0].tolist())
print("Generated text:", generated_text)